<a href="https://colab.research.google.com/github/yjyjy131/Study_Deep_Learning/blob/main/Deep-Learning-Zero-To-All-PyTorch/lab_10_5_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##lab-10-5 VGG

In [1]:
!pip install visdom

     |████████████████████████████████| 676 kB 26.2 MB/s 
     |████████████████████████████████| 54 kB 1.3 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=781c651fb1cb01a465f7d18e7801cbe715b700ea591de54190daaf277426d495
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=189b2383519edf5c8033c6df645163c1f638808f697e5833825fe35a396c8454
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import visdom

In [32]:
import matplotlib.pyplot as plt
import numpy as np

### define loss tracker

In [34]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [35]:
def imshow(img) :
  img = img / 2 + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()
# vis.images(images/2 + 0.5)
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

###make VGG16

In [36]:
import torchvision.models.vgg as vgg

In [37]:
cfg = [32,32,'M', 64,64,128,128,128,'M',256,256,256,512,512,512,'M'] 

In [38]:
class VGG(nn.Module):

  def __init__(self, features, num_classes=100, init_weights=True):
    # 상속 오버로딩, 부모 클래스의 init 메소드 호출
    super(VGG, self).__init__()
    self.features = features
    self.classifier = nn.Sequential(
        # nn.Linear(in_features, out_features)
        nn.Linear(512*4*4, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, num_classes)
    )
    if init_weights: self._initialize_weights()

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

  def _initialize_weights(self) :
    # modules() : 모델 내 정의된 layer 들을 iterable 하게 반환
    for m in self.modules():
      # isinstance() 모듈 내 Layer 가 어떤 형태인지 Conv2d, Liner... 
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None :
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d) :
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

In [39]:
vgg16= VGG(vgg.make_layers(cfg),10,True)

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [41]:
a=torch.Tensor(1, 3, 32, 32).to(device)
out = vgg16(a)
print(out)

tensor([[ 0.0092,  0.0074, -0.0076, -0.0131, -0.0185,  0.0076,  0.0020, -0.0105,
          0.0069, -0.0007]], grad_fn=<AddmmBackward0>)


In [42]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.005, momentum=0.9)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3)

In [ ]:
# loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

### Training

In [ ]:
epochs = 50

for epoch in range(epochs):  
    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # to(device) : GPU 를 사용해서 새로운 copy 를 만들어냄
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [46]:
dataiter = iter(testloader)
images, labels = dataiter.next()
ouputs = vgg16(images.to(device))

In [47]:
# torch.max(input, dimension)
_, predicted = torch.max(outputs, 1)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    outputs = vgg16(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()